# Bond Future Parametrization

This notebook demonstrates how to access and use the **Bond Future** functionalities within the **LSEG Financial Analytics SDK**.

**You will be able to:**
- Define Bond Future with 2 underlying bonds
- Evaluate Bond Future and the underlying bonds analytics

## Imports


Import the following necessary modules:

- `lseg_analytics.pricing.instruments.bond_future` - for Bond Future instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import bond_future as bf

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Bond Repo instrument you need to follow a structured 4-step process:

1. **Bond Definition** - Specify the underlying Bond parameters 
2. **Future Definition** - Specify the Future parameters
3. **Future Definition Instrument** - Create the instrument object using the Future Definition
3. **Pricing Preferences** - Configure pricing parameters, optional


In [2]:
# 1.Define an underlying Bond instrument
bond_definition_1 = bf.BondDefinition(
    notional_ccy = "EUR",
    issue_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),
    end_date = dt.datetime.strptime("2030-01-01", "%Y-%m-%d"),
    fixed_rate_percent = 2,
    interest_payment_frequency = bf.InterestPaymentFrequencyEnum.QUARTERLY,
    interest_calculation_method = bf.InterestCalculationMethodEnum.DCB_ACTUAL_ACTUAL
)
bond_definition_2 = bf.BondDefinition(
    notional_ccy = "EUR",
    issue_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),
    end_date = dt.datetime.strptime("2035-01-01", "%Y-%m-%d"),
    fixed_rate_percent = 3,
    interest_payment_frequency = bf.InterestPaymentFrequencyEnum.QUARTERLY,
    interest_calculation_method = bf.InterestCalculationMethodEnum.DCB_ACTUAL_ACTUAL
)
underlying_bond_1 = bf.BondFutureUnderlyingContract(
        instrument_definition = bond_definition_1,
        instrument_type = "Bond"
)
underlying_bond_2 = bf.BondFutureUnderlyingContract(
        instrument_definition = bond_definition_2,
        instrument_type = "Bond"
)
print("1 - Underlying Bonds definition created")

# 2.Define Future instrument
future_definition = bf.BondFutureDefinition(
    instrument_code = "FOATc1",  # Mandatory field, RIC of the bond future
    underlying_instruments = [underlying_bond_1, underlying_bond_2],
    notional_amount = 2000000  # Override notional amount
)
print("2 - Future instrument defined")

# 3.Create the Future Instrument from the defintion
future_instrument = bf.BondFutureDefinitionInstrument(definition = future_definition)
print("3 - Future Instrument created")

# 4. Configure pricing parameters
pricing_params = bf.BondFuturePricingParameters(
    valuation_date = dt.datetime.strptime("2025-07-18", "%Y-%m-%d"),                
)
print("4 - Pricing parameters configured")

1 - Underlying Bonds definition created
2 - Future instrument defined
3 - Future Instrument created
4 - Pricing parameters configured


## Request Execution

In [3]:
#  Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

# Execute the calculation using the price() function with error handling
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = bf.price(
        definitions=[future_instrument],
        pricing_preferences=pricing_params
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Bond Future pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Bond Future pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instument and Underlying Bond definitions set above. 

 - **pricingPreferences**: Valuation date.

 - **analytics**:
   - **tabularData**: Customizable list of analytics given in `fields` argument of price() function (`data`, `headers`, `statuses`). If not specified all fields are returned.
   - **description**: Instrument summary and also fields not specified with the defaulted values used in the pricing.
   - **nominalMeasures**: list of Bond future sensitivities
   - **pricingAnalysis**: list of analytics used in the pricing
   - **valuation**: Market valuation details on Bond future
   - **error**: root cause of the issue if any

We give below the details on description, pricing analysis, nominal measures and valuation

In [4]:
# Access the description object
description = response.data.analytics[0].description
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentCode": "FOATc1",
    "instrumentDescription": "FOAT DEC5",
    "endDate": "2025-12-08T00:00:00Z",
    "deliveryStartDate": "2025-12-10",
    "deliveryEndDate": "2025-12-10",
    "notionalCcy": "EUR",
    "notionalAmount": 2000000.0,
    "contractSize": 100000.0,
    "couponRatePercent": 6.0
}


In [5]:
# Access the pricing analysis object
pricing_analysis = response.data.analytics[0]["pricingAnalysis"]
df_pricing_analysis = pd.DataFrame(list(pricing_analysis.items()), columns=["Fields", "Value"])
display(df_pricing_analysis.head(9))

,Fields,Value
0,marketDataDate,2025-07-18T00:00:00Z
1,valuationDate,2025-07-18T00:00:00Z
2,cheapestToDeliver,Bond#1
3,conversionFactor,0.789127
4,price,122.84
5,fairPrice,132.174262
6,invoicePrice,96.936361
7,yieldPercent,3.284616
8,impliedYieldPercent,2.785072


In [6]:
# Access the first underlying bond pricing analysis object
bond_1_pricing_analysis = response.data.analytics[0]["pricingAnalysis"]["deliveryBasket"][0]
df_bond_1_pricing_analysis = pd.DataFrame(list(bond_1_pricing_analysis.items()), columns=["Fields", "Value"])
display(df_bond_1_pricing_analysis.head(10))

,Fields,Value
0,isCheapestToDeliver,True
1,instrumentCode,Bond#1
2,couponRatePercent,2.0
3,endDate,2030-01-01T00:00:00Z
4,cleanPrice,104.285474
5,dirtyPrice,104.377865
6,price,104.285474
7,accrued,0.092391
8,yieldPercent,1.0
9,modifiedDuration,4.264036


In [7]:
# Access the second underlying bond pricing analysis object
bond_2_pricing_analysis = response.data.analytics[0]["pricingAnalysis"]["deliveryBasket"][1]
df_bond_2_pricing_analysis = pd.DataFrame(list(bond_2_pricing_analysis.items()), columns=["Fields", "Value"])
display(df_bond_2_pricing_analysis.head(10))

,Fields,Value
0,isCheapestToDeliver,False
1,instrumentCode,Bond#1
2,couponRatePercent,3.0
3,endDate,2035-01-01T00:00:00Z
4,cleanPrice,117.719364
5,dirtyPrice,117.857951
6,price,117.719364
7,accrued,0.138587
8,yieldPercent,1.0
9,modifiedDuration,8.361437


In [8]:
# Access the nominal measures object
nominal_measures = response.data.analytics[0]["nominalMeasures"]
df_nominal_measures = pd.DataFrame(list(nominal_measures.items()), columns=["Fields", "Value"])
display(df_nominal_measures)

,Fields,Value
0,convexity,76.553562
1,dv01Bp,9.569231
2,duration,8.045876
3,modifiedDuration,7.789996


In [9]:
valuation = response.data.analytics[0].valuation
df_valuation = pd.DataFrame(list(valuation.items()), columns=["Fields", "Value"])
display(df_valuation)

,Fields,Value
0,marketValueInDealCcy,2456800.0
1,marketValueInReportCcy,2456800.0
